In [3]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [4]:
data_usd_tr = pd.read_csv("USD_TRY.csv")
data_usd_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   2862 non-null   object
 1   Şimdi   2862 non-null   object
 2   Açılış  2862 non-null   object
 3   Yüksek  2862 non-null   object
 4   Düşük   2862 non-null   object
 5   Hac.    672 non-null    object
 6   Fark %  2862 non-null   object
dtypes: object(7)
memory usage: 156.6+ KB


In [5]:
df = data_usd_tr.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [6]:
# We need to change from "," to "." because it will be a float.
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df

,Tarih,Şimdi
0,19.12.2022,18.6509
1,18.12.2022,18.6393
2,16.12.2022,18.6343
3,15.12.2022,18.6368
4,14.12.2022,18.6232
...,...,...
2857,06.01.2012,1.8776
2858,05.01.2012,1.8830
2859,04.01.2012,1.8785
2860,03.01.2012,1.8730


## LSTM

In [8]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[18.6509 18.6393 18.6343 18.6368 18.6232] 18.575
[18.6393 18.6343 18.6368 18.6232 18.575 ] 18.6325
[18.6343 18.6368 18.6232 18.575  18.6325] 18.6503
[18.6368 18.6232 18.575  18.6325 18.6503] 18.6324
[18.6232 18.575  18.6325 18.6503 18.6324] 18.6355
[18.575  18.6325 18.6503 18.6324 18.6355] 18.624
[18.6325 18.6503 18.6324 18.6355 18.624 ] 18.6227
[18.6503 18.6324 18.6355 18.624  18.6227] 18.6303
[18.6324 18.6355 18.624  18.6227 18.6303] 18.6313
[18.6355 18.624  18.6227 18.6303 18.6313] 18.5895
[18.624  18.6227 18.6303 18.6313 18.5895] 18.6374
[18.6227 18.6303 18.6313 18.5895 18.6374] 18.6166
[18.6303 18.6313 18.5895 18.6374 18.6166] 18.5851
[18.6313 18.5895 18.6374 18.6166 18.5851] 18.5986
[18.5895 18.6374 18.6166 18.5851 18.5986] 18.6062
[18.6374 18.6166 18.5851 18.5986 18.6062] 18.6081
[18.6166 18.5851 18.5986 18.6062 18.6081] 18.6118
[18.5851 18.5986 18.6062 18.6081 18.6118] 18.5889
[18.5986 18.6062 18.6081 18.6118 18.5889] 18.5599
[18.6062 18.6081 18.6118 18.5889 18.5599] 18.592
[18

In [9]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
90/90 [==============================] - 2s 5ms/step - loss: 3.6682
Epoch 2/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0462
Epoch 3/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0497
Epoch 4/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0742
Epoch 5/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0423
Epoch 6/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0451
Epoch 7/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0365
Epoch 8/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0421
Epoch 9/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0584
Epoch 10/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0783
Epoch 11/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0435
Epoch 12/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0452
Epoch 13/200
90/90 [=================

## Daily


In [10]:
print("Prediction: ",round(yhat[0][0],2), "₺ USD/TL") # onluk olarak yukarı yuvarlandı.

Prediction:  18.79 ₺ USD/TL


## Weekly

In [11]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)
yhat_2 = float(yhat_2)
yhat_3 = float(yhat_3)
yhat_4 = float(yhat_4)
yhat_5 = float(yhat_5)


In [12]:
# onluk olarak yukarı yuvarlandı.
print(f"Pazartesi: {round(yhat,3)} Salı: {round(yhat_2,3)} Carsamba: {round(yhat_3,3)} Persembe: {round(yhat_4,3)} Cuma: {round(yhat_5,3)}  ₺ Pound/TRY")


Pazartesi: 18.792 Salı: 19.008 Carsamba: 18.874 Persembe: 18.897 Cuma: 18.886  ₺ Pound/TRY


## Monthly

In [13]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [14]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[18.6509 18.575  18.624  18.6374 18.6081] 18.601999282836914
[18.575  18.624  18.6374 18.6081 18.602 ] 18.528400421142578
[18.624  18.6374 18.6081 18.602  18.5284] 18.56570053100586
[18.6374 18.6081 18.602  18.5284 18.5657] 18.589500427246094
[18.6081 18.602  18.5284 18.5657 18.5895] 18.57939910888672
[18.602  18.5284 18.5657 18.5895 18.5794] 18.587499618530273
[18.5284 18.5657 18.5895 18.5794 18.5875] 18.56679916381836
[18.5657 18.5895 18.5794 18.5875 18.5668] 18.43320083618164
[18.5895 18.5794 18.5875 18.5668 18.4332] 18.30150032043457
[18.5794 18.5875 18.5668 18.4332 18.3015] 18.243000030517578
[18.5875 18.5668 18.4332 18.3015 18.243 ] 18.214599609375
[18.5668 18.4332 18.3015 18.243  18.2146] 18.165000915527344
[18.4332 18.3015 18.243  18.2146 18.165 ] 18.097299575805664
[18.3015 18.243  18.2146 18.165  18.0973] 17.909500122070312
[18.243  18.2146 18.165  18.0973 17.9095] 17.89069938659668
[18.2146 18.165  18.0973 17.9095 17.8907] 17.92530059814453
[18.165  18.0973 17.9095 17.8907 1

In [15]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
18/18 [==============================] - 1s 7ms/step - loss: 13.2841
Epoch 2/300
18/18 [==============================] - 0s 7ms/step - loss: 2.6819
Epoch 3/300
18/18 [==============================] - 0s 7ms/step - loss: 0.5213
Epoch 4/300
18/18 [==============================] - 0s 7ms/step - loss: 0.1793
Epoch 5/300
18/18 [==============================] - 0s 7ms/step - loss: 0.1805
Epoch 6/300
18/18 [==============================] - 0s 7ms/step - loss: 0.1400
Epoch 7/300
18/18 [==============================] - 0s 7ms/step - loss: 0.1723
Epoch 8/300
18/18 [==============================] - 0s 13ms/step - loss: 0.1276
Epoch 9/300
18/18 [==============================] - 0s 12ms/step - loss: 0.1626
Epoch 10/300
18/18 [==============================] - 0s 8ms/step - loss: 0.1270
Epoch 11/300
18/18 [==============================] - 0s 7ms/step - loss: 0.1363
Epoch 12/300
18/18 [==============================] - 0s 9ms/step - loss: 0.1276
Epoch 13/300
18/18 [==============

In [16]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(float(yhat[0][0]),3)}  ₺ USD/TRY ")


Next Month -> Prediction: 18.877  ₺ USD/TRY 
